In [21]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))
import pandas as pd

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import tokenization, removal, stemming

In [9]:
collection = pd.read_csv('../data/TREC_Passage/collection.tsv', sep='\t', names=["pID", "Passage"], header=None)
queries = pd.read_csv('../data/TREC_Passage/queries.dev.tsv', sep='\t', names=["qID", "Query"], header=None)

In [ ]:
#pipeline = Pipeline(collection = '../data/raw/collection.pkl', queries = '../data/raw/queries.pkl')
#pipeline = Pipeline(collection = collection, queries = queries)

In [12]:
collection.Passage

0          The presence of communication amid scientific ...
1          The Manhattan Project and its atomic bomb help...
2          Essay on The Manhattan Project - The Manhattan...
3          The Manhattan Project was the name for a proje...
4          versions of each volume as well as complementa...
                                 ...                        
8841818    When metal salts emit short wavelengths of vis...
8841819    Thousands of people across the United States w...
8841820    The recipe that creates blue, for example, inc...
8841821    On Independence Days of yore, old-timey crowds...
8841822    View full size image. Behind the scenes of the...
Name: Passage, Length: 8841823, dtype: object

In [14]:
queries.head()

,qID,Query
0,1048578,cost of endless pools/swim spa
1,1048579,what is pcnt
2,1048580,what is pcb waste
3,1048581,what is pbis?
4,1048582,what is paysky


In [78]:
x = collection.Passage[:40]

In [79]:
x = x.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))))

100%|██████████| 40/40 [00:00<00:00, 243.15it/s]


In [83]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence

# init embedding
embedding = ELMoEmbeddings()

In [84]:
# embed words in sentence
# create a sentence
import torch
sentence = Sentence('The grass is green .')

# embed words in sentence
embedding.embed(sentence)

input = torch.empty(sentence[0].embedding.size())
tokens_per_sentence = torch.zeros_like(input)

for token in sentence:
    tokens_per_sentence = torch.add(tokens_per_sentence, token.embedding)


In [87]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence
from tqdm import tqdm
import numpy as np
import torch
from nltk.tokenize.treebank import TreebankWordDetokenizer


class Elmo(object):
    def __init__(self):
        # init embedding
        self.embedding = ELMoEmbeddings()

    def transform(self, X):
        elmo_list = []
        for line in tqdm(X):
            detokenized = TreebankWordDetokenizer().detokenize(line)
            sentence = Sentence(detokenized)
            #self.embedding.embed(sentence)
            #sentence = Sentence(line)
            input = torch.empty(sentence[0].embedding.size())
            tokens_per_sentence = torch.zeros_like(input)
            self.embedding.embed(sentence)
            for token in sentence:
                tokens_per_sentence = torch.add(tokens_per_sentence, token.embedding)
            elmo_list.append(tokens_per_sentence)
        return np.array(elmo_list)


In [97]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence
from tqdm import tqdm
import numpy as np
from nltk.tokenize.treebank import TreebankWordDetokenizer
import torch


class Elmo(object):
    def __init__(self):
        # init embedding
        self.embedding = ELMoEmbeddings()

    def transform(self, X):

        elmo_list = []
        for line in tqdm(X):
            detokenized = TreebankWordDetokenizer().detokenize(line)
            sentence = Sentence(detokenized)
            self.embedding.embed(sentence)
            input = torch.empty(sentence[0].embedding.size())
            tokens_per_sentence = torch.zeros_like(input)
            for token in sentence:
                tokens_per_sentence = torch.add(tokens_per_sentence, token.embedding)
            elmo_list.append(tokens_per_sentence)
        return np.array(elmo_list)

In [95]:
el = Elmo()
output = el.transform(x)

100%|██████████| 40/40 [00:17<00:00,  2.23it/s]
c:\users\d073999\documents\priv-dev\retriever\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
c:\users\d073999\documents\priv-dev\retriever\lib\site-packages\ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [96]:
output

array([tensor([-4.9217, -1.4432,  4.2792,  ...,  5.8397, 36.4698,  4.1877]),
       tensor([-3.7379e+00, -2.2561e+00,  1.5099e-02,  ...,  1.5140e+00,
                3.6263e+01, -2.1968e+00])                               ,
       tensor([-0.4364, -1.7102, -3.0806,  ...,  4.9637, 25.7533,  4.8212]),
       tensor([-1.5862, -2.1382, -4.6523,  ...,  4.6862, 35.8381,  4.3871]),
       tensor([ -4.6053,  -4.5531,  -0.9315,  ..., -17.8543,  50.0152,  -4.6643]),
       tensor([-2.9713, -1.6235,  1.1651,  ..., 14.4024, 50.3542,  4.7282]),
       tensor([ 1.2184, -2.4923, -0.3482,  ..., 10.2864, 23.6172, -5.8068]),
       tensor([ -0.8632,  -2.2151,  -2.3177,  ...,  -7.8934,  85.5076, -15.9382]),
       tensor([-1.6970,  1.1990, -0.8303,  ...,  2.7465, 10.9501,  1.6276]),
       tensor([-2.9909e-02,  1.2811e+00,  1.9970e-01,  ..., -1.0623e+01,
                4.7564e+01,  3.6053e+00])                               ,
       tensor([-0.0537, -3.2913,  3.3061,  ..., -8.0761, 42.2549,  2.1429]),
 

In [108]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-cloud -y
!pip install -U tensorflow==1.15

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\cli\base_command.py", line 180, in _main
    status = self.run(options, args)
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\commands\uninstall.py", line 89, in run
    uninstall_pathset.commit()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\req\req_uninstall.py", line 442, in commit
    self._moved_paths.commit()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\req\req_uninstall.py", line 282, in commit
    save_dir.cleanup()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\utils\temp_dir.py", line 184, in cleanup
    rmtree(self._path)
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 339, in wrapped_f
    return self(f, *args, **kw)
  

Looking in indexes: https://int.repositories.cloud.sap/artifactory/api/pypi/build-releases-pypi/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/build-milestones-pypi/simple
  Using cached https://int.repositories.cloud.sap/artifactory/api/pypi/build-milestones-pypi/packages/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=2e79851a5a64d77ddea493df99b35153515a67e6f636e2315774d102362af3c8
  Stored in directory: c:\users\d073999\appdata\local\pip\cache\wheels\72\f2\b3\e50dc833b3e9829b60168ad4921ee7ccbfffe6738204b06a38
Successfully built gast
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Suc

You should consider upgrading via the 'C:\Users\D073999\Documents\Priv-Dev\Retriever\Scripts\python.exe -m pip install --upgrade pip' command.


In [109]:
!pip uninstall pytorch-lightning -y
!pip uninstall tensorflow-probability -y

In [154]:
##Tensorflow Hub ELMO-2
import tensorflow_hub as hub
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["default"]

def create_elmo_embeddings(data):
    embed=elmo(
        tf.constant(data))["elmo"]
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        sess.run(tf.compat.v1.tables_initializer())
        out_x=sess.run(embed)
        #out_y=ses.run(tf.reduce_mean(embed,1))
        return out_x
z=queries.Query.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [228]:
x = collection.Passage[:40]

In [229]:
x

0     The presence of communication amid scientific ...
1     The Manhattan Project and its atomic bomb help...
2     Essay on The Manhattan Project - The Manhattan...
3     The Manhattan Project was the name for a proje...
4     versions of each volume as well as complementa...
5     The Manhattan Project. This once classified ph...
6     Nor will it attempt to substitute for the extr...
7     Manhattan Project. The Manhattan Project was a...
8     In June 1942, the United States Army Corps of ...
9     One of the main reasons Hanford was selected a...
10    group discussions, community boards or panels ...
11    punishment designed to repair the damage done ...
12    Tutorial: Introduction to Restorative Justice....
13    Organize volunteer community panels, boards, o...
14    The purpose of this paper is to point out a nu...
15    Each of these types of communitiesâthe geogr...
16    The approach is based on a theory of justice t...
17    Inherent in many peopleâs understanding 

In [172]:
z=x.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [187]:
def elmo_vectors(x):
  embeddings = []
  for elm in x:
        print(elm)
        e = elmo(elm.tolist())["elmo"]
        embeddings.append(elmo(x.tolist())["elmo"])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [230]:
x = x.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))));x

100%|██████████| 40/40 [00:00<00:00, 367.86it/s]


0     [presenc, commun, amid, scientif, mind, equal,...
1     [manhattan, project, atom, bomb, help, bring, ...
2     [essay, manhattan, project, manhattan, project...
3     [manhattan, project, name, project, conduct, w...
4     [version, volum, well, complementari, websit, ...
5     [manhattan, project, classifi, photograph, fea...
6     [attempt, substitut, extraordinarili, rich, li...
7     [manhattan, project, manhattan, project, resea...
8     [june, 1942, unit, state, armi, corp, engineer...
9     [one, main, reason, hanford, select, site, man...
10    [group, discuss, commun, board, panel, third, ...
11    [punish, design, repair, damag, done, victim, ...
12    [tutori, introduct, restor, justic, restor, ju...
13    [organ, volunt, commun, panel, board, committe...
14    [purpos, paper, point, number, unresolv, issu,...
15    [type, communitiesâth, geograph, commun, vic...
16    [approach, base, theori, justic, consid, crime...
17    [inher, mani, peopleâ, understand, notio

In [244]:
output = []
from nltk.tokenize.treebank import TreebankWordDetokenizer
output = []
for tokens in x:
    r =TreebankWordDetokenizer().detokenize(tokens)
    output.append(r)
output

['presenc commun amid scientif mind equal import success manhattan project scientif intellect cloud hang impress achiev atom research engin success truli meant hundr thousand innoc live obliter',
 'manhattan project atom bomb help bring end world war ii legaci peac use atom energi continu impact histori scienc',
 'essay manhattan project manhattan project manhattan project see make atom bomb possibl success project would forev chang world forev make known someth power manmad',
 'manhattan project name project conduct world war ii develop first atom bomb refer specif period project 194 â\x80¦ 21946 control us armi corp engin administr gener lesli r grove',
 'version volum well complementari websit first websiteâ\x80\x93th manhattan project interact historyâ\x80\x93i avail offic histori heritag resourc websit http wwwcfo doegovme70histori offic histori heritag resourc nation nuclear secur',
 'manhattan project classifi photograph featur first atom bomb â\x80\x94 weapon atom scientist nic

In [245]:
z=output#.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [247]:
elmo_output

array([[[-0.16142987,  0.39164305,  0.2924859 , ..., -0.10307619,
          0.47838593,  0.10405383],
        [ 0.103149  , -0.20897631,  0.40901518, ..., -0.14926252,
          0.6624829 , -0.18022116],
        [-0.24741618,  0.30307624, -0.5055893 , ..., -0.26527908,
          0.17358826, -0.03860291],
        ...,
        [-0.12553898, -0.13239129, -0.04669396, ..., -0.32613608,
          0.5977138 ,  0.13836601],
        [ 0.5781988 , -0.35236633, -0.37354636, ..., -0.38017738,
          0.07846026,  0.09849334],
        [ 0.1008294 , -0.49969184, -0.3491489 , ...,  0.12266825,
          0.16703467,  0.42627513]],

       [[ 0.306329  ,  0.4091187 ,  0.48765004, ...,  0.13162018,
          0.62339723, -0.22805919],
        [ 0.7706266 , -0.03343945,  0.12615913, ...,  0.03431229,
          0.20650187, -0.33816066],
        [-0.01331159,  0.18552707, -0.15152037, ...,  0.21959701,
          0.87772703,  0.6102616 ],
        ...,
        [-0.02840841, -0.04353216,  0.04130162, ...,  

elmo_output